<a href="https://colab.research.google.com/github/rickyjasso/TC2008B/blob/main/SemaforosM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 9.3 MB/s 
     |████████████████████████████████| 60 kB 5.5 MB/s 


In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `Multigrid` que nos deja tener varios agentes en una celda
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import signal

import math

In [ ]:
class Move(Agent):
    #Inicializacion
    grid = None
    x = None
    y = None
    moore = False
    include_center=False

    def __init__(self, unique_id, pos, model, moore=False): #moore es para moverse en 8 direcciones
    
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore
    
    def move(self):
        next_moves = self.model.grid.get_neighborhood(self.pos,moore=False,include_center=False)
        posicion_azar=random.choice(next_moves)
        if self.model.grid.is_cell_empty(posicion_azar):
            self.movimientos += 1

In [ ]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
            grid[x][y] = agent.height
    return grid

class Light(Agent):
  def __init__(self, unique_id, model, direction):
    super().__init__(unique_id, model)
    self.state=1 #rojo = 0, amarillo=1, verde =2
    self.height=2
    self.direction = direction

class Car(Agent):
  def __init__(self, unique_id, model, direction, vuelta, height):
    super().__init__(unique_id, model)
    self.height= height
    self.vuelta= vuelta #0 = izquierda, 1=recto, 2=derecha
    self.is_moving=True 
    self.direction = direction

  def checkBoundaries(self,pos, posNext):
    x, y = pos
    xNext, yNext = posNext
    if (x == 0 and xNext == -1):
      return False
    if (y == 0 and yNext == -1):
      return False
    if (x == self.model.height - 1 and xNext == self.model.height):
      return False
    if (y == self.model.width - 1 and yNext == self.model.width):
      return False
    return True
  
    '''
    0-A, 0-B, 0-C, 0-D
    
    2-A, 2-B, 2-C, 2-D
    '''
  def turnLeft(self):

    x, y = self.pos

    if (self.direction == 'A'):
      if (y != 7):
        if self.checkBoundaries(self.pos, (x,y+1)) == True:
          self.model.grid.move_agent(self, (x,y+1))
      elif self.checkBoundaries(self.pos, (x-1,y)) == True:
        self.model.grid.move_agent(self, (x-1,y))
        
    if (self.direction == 'B'):
      if (x != 6):
        if self.checkBoundaries(self.pos, (x-1,y)) == True:
          self.model.grid.move_agent(self, (x-1,y))
      elif self.checkBoundaries(self.pos, (x,y-1)) == True:
        self.model.grid.move_agent(self, (x,y-1))

    if (self.direction == 'C'):
      if (y != 6):
        if self.checkBoundaries(self.pos, (x,y-1)) == True:
          self.model.grid.move_agent(self, (x,y-1))
      elif self.checkBoundaries(self.pos, (x+1,y)) == True:
        self.model.grid.move_agent(self, (x+1,y))

    if (self.direction == 'D'):
      if (x != 7):
        if self.checkBoundaries(self.pos, (x+1,y)) == True:
          self.model.grid.move_agent(self, (x+1,y))
      elif self.checkBoundaries(self.pos, (x,y+1)) == True:
        self.model.grid.move_agent(self, (x,y+1))

  def turnRight(self):

    x, y = self.pos

    if (self.direction == 'A'):
      if (y != 5):
        if self.checkBoundaries(self.pos, (x,y+1)) == True:
          self.model.grid.move_agent(self, (x,y+1))
      elif self.checkBoundaries(self.pos, (x+1,y)) == True:
        self.model.grid.move_agent(self, (x+1,y))
        
    if (self.direction == 'B'):
      if (x != 8):
        if self.checkBoundaries(self.pos, (x-1,y)) == True:
          self.model.grid.move_agent(self, (x-1,y))
      elif self.checkBoundaries(self.pos, (x,y+1)) == True:
        self.model.grid.move_agent(self, (x,y+1))

    if (self.direction == 'C'):
      if (y != 8):
        if self.checkBoundaries(self.pos, (x,y-1)) == True:
          self.model.grid.move_agent(self, (x,y-1))
      elif self.checkBoundaries(self.pos, (x-1,y)) == True:
        self.model.grid.move_agent(self, (x-1,y))

    if (self.direction == 'D'):
      if (x != 5):
        if self.checkBoundaries(self.pos, (x+1,y)) == True:
          self.model.grid.move_agent(self, (x+1,y))
      elif self.checkBoundaries(self.pos, (x,y-1)) == True:
        self.model.grid.move_agent(self, (x,y-1))

  def turnStraight(self):
    x, y = self.pos
    if (self.direction == 'A'):
      if self.checkBoundaries(self.pos, (x,y+1)) == True:
        self.model.grid.move_agent(self, (x,y+1))
    if (self.direction == 'B'):
      if self.checkBoundaries(self.pos, (x-1,y)) == True:
        self.model.grid.move_agent(self, (x-1,y))
    if (self.direction == 'C'):
      if self.checkBoundaries(self.pos, (x,y-1)) == True:
        self.model.grid.move_agent(self, (x,y-1))
    if (self.direction == 'D'):
      if self.checkBoundaries(self.pos, (x+1,y)) == True:
        self.model.grid.move_agent(self, (x+1,y))          


  def brakeLight(self):
    neighbours = self.model.grid.get_neighbors(
      self.pos, moore=True, include_center=False,)
    for neighbor in neighbours:
      if isinstance(neighbor, Light) and neighbor.direction == self.direction:
        self.is_moving = False


  def move(self):
    if self.vuelta == 0:
      self.turnLeft()         
    if self.vuelta == 1:
      self.turnStraight()          
    if self.vuelta == 2:
      self.turnRight()

  def step(self):
    if self.is_moving == False:
      self.is_moving = True
    else:
      self.brakeLight()

  def advance(self):
    if self.is_moving: 
      self.move()

class Street(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.height = 1
      

class IntersectionModel(Model):
  def __init__(self, width, height):
    self.grid = MultiGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.width = width
    self.height = height
    self.CarsA=list(((7,0),(8,0)))
    CarsB=list(((13,7),(13,8)))
    CarsC=list(((5,13),(6,13)))
    CarsD=list(((0,5),(0,6)))
    #Cars=list(((7,0),(8,0)))
    Lights=list(((8,5, ),(8,8),(5,8),(5,5)))
    #Waypoints=list(((w1,w2),(x1,x2),(y1,y2),(z1,z2)))

    Directions=list(('A','B','C','D'))
    #Street
    StreetPos = [
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0]
    ]
    for (content, x, y) in self.grid.coord_iter():
      if StreetPos[x][y] == 1:
        w = Street((x, y), self)
        self.grid.place_agent(w, (x,y)) 

    #Cars
    '''
    for z in range(len(CarsA)):
      x, y = CarsA[z]
      if (x == 7):
        vuelta = np.random.choice([0,1])
      elif (x == 8):
        vuelta = np.random.choice([1,2])
      c = Car(CarsA[z], self, 'A', vuelta, 3)
      self.grid.place_agent(c, CarsA[z])   
      self.schedule.add(c)

    
    for z in range(len(CarsB)):
      x, y = CarsB[z]
      if (y == 7):
        vuelta = np.random.choice([0,1])
      elif (y == 8):
        vuelta = np.random.choice([1,2])
      c = Car(CarsB[z], self, 'B', vuelta, 4)
      self.grid.place_agent(c, CarsB[z]) 
      self.schedule.add(c)

    for z in range(len(CarsC)):
      x, y = CarsC[z]
      if (x == 6):
        vuelta = np.random.choice([0,1])
      elif (x == 5):
        vuelta = np.random.choice([1,2])
      c = Car(CarsC[z], self, 'C', vuelta, 5)
      self.grid.place_agent(c, CarsC[z]) 
      self.schedule.add(c)

    for z in range(len(CarsD)):
      x, y = CarsD[z]
      if (y == 6):
        vuelta = np.random.choice([0,1])
      elif (y == 5):
        vuelta = np.random.choice([1,2])
      c = Car(CarsD[z], self, 'D', vuelta, 6)
      self.grid.place_agent(c, CarsD[z]) 
      self.schedule.add(c)

      '''
             
    #semaforos
    for z in range(4):
      l = Light(Lights[z], self, Directions[z])
      self.grid.place_agent(l, Lights[z]) 
      self.schedule.add(l)
      
    # Aquí definimos con colector para obtener el grid completo.
    self.datacollector = DataCollector(model_reporters={"Grid": get_grid})

  def step(self):
    z = np.random.choice([0,1])
    x, y = self.CarsA[z]
    if (x == 7):
      vuelta = np.random.choice([0,1])
    elif (x == 8):
      vuelta = np.random.choice([1,2])
    c = Car((self.CarsA[z], np.random), self, 'A', vuelta, 3) # CREAR RANDOM DEL 1 AL 1000
    self.grid.place_agent(c, self.CarsA[z])   
    self.schedule.add(c)
    self.datacollector.collect(self)
    self.schedule.step()

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 14


# Max RunTime
start_time = time.time()
maxTime = .02

frameCounter = 0

model = IntersectionModel(GRID_SIZE, GRID_SIZE)
while(time.time() - start_time) < maxTime:
  frameCounter+=1
  model.step()

frameCounter+=1
model.step()


# Tiempo de Ejecucion
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))


# Numero de Movimientos Realizados por los Agentes
print('Numero de Movimientos Realizados: ', frameCounter)


Exception: ignored

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(all_grid.iloc[0][0])

[[0. 0. 0. 0. 0. 6. 6. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 5.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 5.]
 [3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [3. 1. 1. 1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 4. 4. 0. 0. 0. 0. 0.]]


In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Greens)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=frameCounter)


In [ ]:
anim